# **Collecting parking lot occupancy data by vehicles**

In [1]:
SIMULATION_BURNIN = "../02_scenario/burnin.sumocfg"
SIMULATION_STABLE = "../02_scenario/stable.sumocfg"
PARKING_DEFINITION = "../02_scenario/parking_areas.add.xml"
SEEDS = ['42', '1234', '1867', '613', '1001']

SENSOR_RANGE = 50 #[m]

OUTPUT_DIR = "../04_results/"

In [2]:
import json
import numpy as np
from tqdm.notebook import trange

import pandas as pd

In [3]:
import os, sys
import traci

SUMO_HOME = os.environ["SUMO_HOME"] #locating the simulator
sys.path.append(SUMO_HOME+"/tools")

import sumolib
from importlib import reload
import traci.constants as tc

In [4]:
parking_data = pd.read_xml(PARKING_DEFINITION, xpath="parkingArea")
parking_data = parking_data.set_index(keys=["id"])

parking_capacities = {}
for i,r in parking_data.iterrows():
    parking_capacities[r.name] = r.roadsideCapacity

In [5]:
def run_simulation(seed):
    #burn in with current seed (SUMO handles saving the resulting state):
    #(command line execution is way faster than TraCI)
    !sumo -c {SIMULATION_BURNIN} --seed {seed} --no-step-log
    
    #running stable simulation and obtaining results by TraCI:
    sumo_cmd = ["sumo", "-c", SIMULATION_STABLE, "--seed", str(seed), "--no-step-log"]
    traci.start(sumo_cmd)

    p_ids = traci.parkingarea.getIDList()

    vehicle_data = pd.DataFrame(columns=["meas_id", "veh_id", "parking_id", "time", "occupancy"])
    active_vehicles = set()
    measurement_list = []

    end_time = traci.simulation.getEndTime()
    timestep = 0
    while ((end_time > 0) and (timestep < end_time)) or ((end_time<0) and (traci.simulation.getMinExpectedNumber()>0)):
        traci.simulationStep()
        timestep = traci.simulation.getTime()
        #if timestep%(60*60) == 0:
            #print(f'{timestep//(60*60)-96:2d}:00 ({len(active_vehicles)})')

        #new cars are added:
        new_vehicles = traci.simulation.getDepartedIDList()
        for veh in new_vehicles:
            active_vehicles.add(veh)
            try:
                traci.vehicle.subscribeContext(veh, tc.CMD_GET_PARKINGAREA_VARIABLE, SENSOR_RANGE, [tc.VAR_NAME])
            except:
                pass
        #stop ending (restarting) vehicles:
        starting_vehicles = traci.simulation.getStopEndingVehiclesIDList()
        for veh in starting_vehicles:
            active_vehicles.add(veh)
            try:
                traci.vehicle.subscribeContext(veh, tc.CMD_GET_PARKINGAREA_VARIABLE, SENSOR_RANGE, [tc.VAR_NAME])
            except:
                pass
        #stopping vehicles:
        stopping_vehicles = traci.simulation.getStopStartingVehiclesIDList()
        for veh in stopping_vehicles:
            if veh in active_vehicles:
                active_vehicles.remove(veh)
                try:
                    traci.vehicle.unsubscribeContext(veh, tc.CMD_GET_PARKINGAREA_VARIABLE, SENSOR_RANGE)
                except:
                    pass
        #arriving vehicles:
        arriving_vehicles = traci.simulation.getArrivedIDList()
        for veh in arriving_vehicles:
            if veh in active_vehicles:
                active_vehicles.remove(veh)

        parking_occupancies = {}
        for p_id in parking_data.index:
            parking_occupancies[p_id] = traci.parkingarea.getVehicleCount(p_id) / parking_capacities[p_id]

        for veh in active_vehicles:
            p_ids = traci.vehicle.getContextSubscriptionResults(veh)

            for p in p_ids:
                occupancy = parking_occupancies[p]

                new_occup_measurement = {"veh_id": veh,
                                    "parking_id": p,
                                    "time": timestep,
                                    "occupancy": occupancy}
                measurement_list.append(new_occup_measurement)

    traci.close()
    vehicle_data = pd.DataFrame.from_records(measurement_list)
    vehicle_data = vehicle_data.drop_duplicates()
    vehicle_data.to_csv(OUTPUT_DIR+f'poccup_by_vehs_{seed}.csv', index=False)

In [6]:
from multiprocessing import Pool

#simulation in parallel:
with Pool(len(SEEDS)) as pool:
    pool.map(run_simulation, SEEDS)

Error: Vehicle 'h500c1:0' could not assign stop 'parkingArea:pa-27' after rerouting (pa64:parkingAreaReroute) at time=38384.00.
Error: Vehicle 'h500c1:0' could not assign stop 'parkingArea:pa-60' after rerouting (pa64:parkingAreaReroute) at time=38384.00.
Error: Vehicle 'h500c1:0' could not assign stop 'parkingArea:pa-27' after rerouting (pa64:parkingAreaReroute) at time=38384.00.
Error: Vehicle 'h500c1:0' could not assign stop 'parkingArea:pa-60' after rerouting (pa64:parkingAreaReroute) at time=38384.00.
Error: Vehicle 'h500c1:0' could not assign stop 'parkingArea:pa-27' after rerouting (pa64:parkingAreaReroute) at time=38384.00.
Error: Vehicle 'h500c1:0' could not assign stop 'parkingArea:pa-60' after rerouting (pa64:parkingAreaReroute) at time=38384.00.
Error: Vehicle 'h500c1:0' could not assign stop 'parkingArea:pa-27' after rerouting (pa64:parkingAreaReroute) at time=38384.00.
Error: Vehicle 'h500c1:0' could not assign stop 'parkingArea:pa-60' after rerouting (pa64:parkingAreaRer

 Retrying in 1 seconds
 Retrying in 1 seconds
 Retrying in 1 seconds


 Retrying in 1 seconds


Error: Answered with error to command 0x84: Vehicle 'h23c2:0' is not known.
Error: Answered with error to command 0x84: Vehicle 'h293c1:0' is not known.
